In [1]:
#import

import os
import glob
from dotenv import load_dotenv
import gradio as gr

/Users/triimamwicaksono/Document/Mini Project/my-digital-twin/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [127]:
#import langchain

from langchain.document_loaders import DirectoryLoader, TextLoader,PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.memory import ChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [3]:
def load_docs(data_dir: str) -> list:
    """
    Load all PDF and MD files from a folder into LangChain Documents.
    Returns a list of Documents with basic metadata.
    """
    documents = []

    for file in os.listdir(data_dir):
        if file.endswith(".pdf"):
            loader = PyMuPDFLoader(os.path.join(data_dir, file))
            documents.extend(loader.load())
        elif file.endswith(".md"):
            loader = TextLoader(os.path.join(data_dir, file), encoding='utf-8')
            documents.extend(loader.load())

    return documents


In [4]:
def chunk_docs(docs:list) -> list:
    """Chunk documents into smaller pieces."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
    return text_splitter.split_documents(docs)

In [5]:
docs = load_docs("./knowledge-base")
chunks = chunk_docs(docs)
    

In [105]:
print("Total chunks:", len(chunks))
print("First chunk:\n", chunks[0].page_content[:500])

Total chunks: 10
First chunk:
 Senior Data Analyst with 6+ years of experience in product experimentation, behavioral analytics, and
KPI optimization to drive product-led growth. Proven track record in shaping roadmap decisions
through statistically sound A/B testing, funnel analysis, and LTV/retention modeling. Highly skilled in
building scalable data pipelines and real-time dashboards that influence executive and cross-
functional decisions. Currently pursuing a Master’s in Business Information Technology (specializing in
D


In [7]:
db_name = "chroma-db"

# Ensure environment variables are loaded
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
	raise ValueError("OPENAI_API_KEY environment variable not found. Please set it in your .env file.")

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

if not os.path.exists(db_name):
    os.makedirs(db_name)
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

    vectordb.persist()
else:
    vectordb = Chroma(persist_directory=db_name, embedding_function=embeddings)
    vectordb.persist()



/var/folders/6x/tjtz85yd37n5sb4nwqmv5wc40000gn/T/ipykernel_89960/3185160268.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)
/var/folders/6x/tjtz85yd37n5sb4nwqmv5wc40000gn/T/ipykernel_89960/3185160268.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [128]:
llm = ChatOpenAI(temperature=0.7, model = "gpt-4o")
prompt = ChatPromptTemplate.from_messages([
        ("system",
        "You are Tri's representative. Answer ONLY from the provided context. "
        "If you don't know, say \"I don't know\" and ask the user to clarify. "
        "Keep answers to 2–4 sentences. Reply in the same language as the question."
        "Context:\n{context}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "Question:{input}")])


documents_chain = create_stuff_documents_chain(llm,prompt)

retriever = vectordb.as_retriever(search_kwargs={"k": 15})

retrieval_chain = create_retrieval_chain(retriever, documents_chain)
retrieval_chain = retrieval_chain | RunnableLambda(lambda x: x["answer"]) | StrOutputParser()





                                                 


In [129]:

store = {}

def get_session_history(user_id: str, conversation_id: str) -> ChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


chain_with_history = RunnableWithMessageHistory(retrieval_chain,
                                                history_messages_key="history",
                                                input_messages_key="input",
                                                get_session_history=get_session_history,
                                                history_factory_config=[
                                                ConfigurableFieldSpec(
                                                    id="user_id",
                                                    annotation=str,
                                                    name="User ID",
                                                    description="Unique identifier for the user.",
                                                    default="",
                                                    is_shared=True,
                                                ),
                                                ConfigurableFieldSpec(
                                                    id="conversation_id",
                                                    annotation=str,
                                                    name="Conversation ID",
                                                    description="Unique identifier for the conversation.",
                                                    default="",
                                                    is_shared=True,
                                                ),
                                            ])

In [130]:
cfg = {"configurable": {"user_id": "u1", "conversation_id": "c1"}}
result = chain_with_history.invoke({"input": "What did Tri do at Tokopedia?"}, config=cfg)
print(result)


At Tokopedia, Tri worked as a Senior Data Analyst from 2018 to 2022. He partnered with Product Managers, Engineers, and Designers to drive experimentation and product optimization at scale. He designed and executed over 50 A/B tests, which led to a 10% uplift in checkout conversion rates, and reduced time-to-insight by automating experimentation reporting and metric monitoring. Tri also built SQL pipelines and maintained dashboards to improve data-driven decision-making across the platform.


In [131]:
def get_config(session_hash:str):
    return {"configurable": {"user_id": session_hash, "conversation_id": session_hash}}


In [ ]:
def respond(message, history, session_id):
    cfg = {"configurable": {"user_id": session_id, "conversation_id": session_id}}

    partial = ""
    for chunk in chain_with_history.stream({"input": message}, config=cfg):
        if isinstance(chunk, dict):
            text = chunk.get("answer", "")
        else:
            text = getattr(chunk, "content", str(chunk))

        partial += text
        yield partial   # just the assistant message being built


In [ ]:
import time
import gradio as gr

def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.3)  # simulate delay
        yield "You typed: " + message[: i+1]   # stream partial output

gr.ChatInterface(
    fn=slow_echo, 
    type="messages"
).launch()


* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
